# Programming differential equations in Julia

Julia has a rich ecosystem for modelling based on differential equations. The [SciML](https://sciml.ai/) project provides a convenient entry point.

The ecosystem provides different tools to define and solve differential equations, as well as explore the solutions obtained.

### Example 1

Consider the equation $$\frac{d}{dt}x(t) = \alpha x(t).$$

Given an initial condition $x(0) = x_0$, it can be verified that the solution to the initial value problem is given by $$x(t) = x_0 e^{\alpha t}.$$

In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

@variables t x(t)=0.5 # specify symbolically the variables involved together with the initial condition
@parameters α=0.2 # define parameters

# the symbolic differentiation operator
D = Differential(t) 

# define the equation(s)
eqs = [D(x) ~ α*x]

# construct a symbolic system
@named sys = ODESystem(eqs, t)

# define the timespan of the simulation
tspan = (0.0, 10.0) 

# convert to ODE problem from symbolic form
prob = ODEProblem(sys, [], tspan) 

# Solve the ODE problem
sol = solve(prob) 

# play around with the result
plot(sol, title = "Linear ODE")

In [ ]:
sol(0) # value of solution at 0

In [ ]:
sol(10) # value of solution at 10

In [ ]:
sol # note the form of u (a vector of vectors), as well as the tabulation grid

In [ ]:
# here is how to tabulate over a pre-specified grid
sol = solve(prob, saveat = range(tspan[1], stop = tspan[2], length = 100)) 

A simpler (and perhaps slightly outdated) way of accomplishing the same.

In [ ]:
using DifferentialEquations

function lin_ode!(du, u, p, t)
    du[1] = p[1] * u[1]
end

u0 = [0.5]
p = [0.2]
tspan = (0.0, 10.0)  # Time interval for solution
npoints = 100        # Number of output points

prob1 = ODEProblem(lin_ode!, u0, tspan, p)
sol1 = solve(prob1, Tsit5()) # note that the solver has been explicitly specified here

plot(sol1)


In [ ]:
sol1

### Example 2 (the Lotka-Volterra predator-prey model, from the documentation)

We consider the interaction between a prey species with population density $x(t)$ and a predator species with population density $y(t)$. The parameter $\alpha$ is the birth rate of the prey species, $\gamma$ is the death rate of the predator species and the parameters $\beta$ and $\delta$ describe the effects of interaction between species.

$$ \frac{dx}{dt} = \alpha x - \beta x y $$
$$ \frac{dy}{dt} = -\gamma y + \delta x y $$

In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

# Define our state variables: state(t) = initial condition
@variables t x(t)=1 y(t)=1 z(t)=2

# Define our parameters
@parameters α=1.5 β=1.0 γ=3.0 δ=1.0

# Define our differential: takes the derivative with respect to `t`
D = Differential(t)

# Define the differential equations
# Note the last element, which is an algebraic equation counting the total number of animals
eqs = [D(x) ~ α * x - β * x * y
       D(y) ~ -γ * y + δ * x * y
       z ~ x + y] 

# Bring these pieces together into an ODESystem with independent variable t
@named sys = ODESystem(eqs, t)

# Symbolically Simplify the System (removes the algebraic equation)
simpsys = structural_simplify(sys)

# Convert from a symbolic to a numerical problem to simulate
tspan = (0.0, 10.0)
prob = ODEProblem(simpsys, [], tspan)

# Solve the ODE
sol = solve(prob)

# Plot the solution
p1 = plot(sol, title = "Rabbits vs Wolves")
p2 = plot(sol, idxs = z, title = "Total Animals")

plot(p1, p2, layout = (2, 1))

# The augmented Solow growth model

### Model setup

- Closed economy, continuous time
- Main variables: output $Y$, capital $K$, labour $L$ and technological progress $A$
- Labour-augmenting production technology:
  $$Y(t) = F(K(t),A(t)L(t))$$
- Assumptions about the production function:
    - $F(0,0)=0$
    - Positive marginal products, law of diminishing returns
    $$F'_{x_i} > 0, \quad F''_{x_i x_i} < 0$$
    - Constant returns to scale
    $$F(cK,cAL) = cF(K,AL),~\forall c \geq 0$$
    - Inada conditions
    $$\lim_{x_i \rightarrow 0}F'_{x_i} = +\infty,\quad \lim_{x_i \rightarrow +\infty}F'_{x_i} = 0$$
- Variables per unit of *effective* labour:
    $$c=\frac{C}{AL},~y=\frac{Y}{AL},~k=\frac{K}{AL}$$
- Production function in intensive form
  $$\frac{1}{AL}F\left(K,AL \right) = F\left( \frac{K}{AL},1 \right) =: f(k) \qquad \Rightarrow \qquad y=f(k)$$
- Properties of $f(k)$
    - $f(0)=0$
    - $f'(k)>0, \quad f''(k)<0 $
    - $\lim_{k \rightarrow 0}f'_{k} = +\infty,\quad \lim_{k \rightarrow +\infty}f'_{k} = 0$


**Example** (Cobb-Douglas production function): If $F(K,AL) = K^\alpha (AL)^{1-\alpha},~~ 0<\alpha<1$, then $f(k)=k^\alpha$.


- Exogenous growth dynamics
  $$\dot{L}(t) = n L(t) \qquad \Rightarrow \qquad L(t) = L(0)e^{nt}$$
  $$\dot{A}(t) = g A(t) \qquad \Rightarrow \qquad A(t) = A(0)e^{gt}$$
  
  Here $n$ is the constant population growth rate and $g$ is the constant technology growth rate.
- Capital accumulation
    $$\dot{K}(t) = sY(t)-\delta K(t),$$
    where $s \in (0,1)$ is the saving rate and $\delta \in (0,1)$ is the depreciation rate.
    

### The main equation

\begin{array}[lll] $\dot{k}(t) & = &   \dfrac{\dot{K}(t)}{A(t)L(t)} - \dfrac{K(t)}{(A(t)L(t))^2}[A(t)\dot{L}(t)+L(t)\dot{A}(t)] \\
{} & = & \dfrac{\dot{K}(t)}{A(t)L(t)} - \dfrac{K(t)}{A(t)L(t)} \dfrac{\dot{L}(t)}{L(t)} - \dfrac{K(t)}{A(t)L(t)} \dfrac{\dot{A}(t)}{A(t)} \\ {} & = &  \dfrac{sY(t) - \delta K(t)}{A(t)L(t)} - k(t)n - k(t) g \\ {} & = & s \dfrac{sY(t)}{A(t)L(t)} - \delta k(t) - n k(t) - g k(t) \\ {} & = & s f(k(t)) - (n+g+\delta)k(t).
\end{array}

### Steady state

$$\dot{k}(t) = 0 \qquad \Rightarrow \qquad s f(k^*) = (n+g+\delta)k^* $$

Clearly we have the trivial steady state $k^* = 0$. There is one more steady state $k^* > 0$.

### Coding the Solow model

In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

@variables t k(t)=1.0 
@parameters α=0.6 δ=.08 n=0.03 g=0.02 s=0.2

f(k) = k^α

D = Differential(t)


eqs = [D(k) ~ s*f(k) - (n + g + δ)*k] 

@named sys = ODESystem(eqs, t);

tspan = (0.0, 100.0)
prob = ODEProblem(sys, [], tspan)

sol = solve(prob)

plot(sol)

In [ ]:
# A hacky way to compute and display the steady state

using ModelingToolkit, DifferentialEquations, Plots, Roots

alpha=0.6
delta=0.08
nn = 0.03
gg = 0.02
ss = 0.2

@variables t k(t)=10.0 
@parameters α=alpha δ=delta n=nn g=gg s=ss

f(k) = k^alpha

D = Differential(t)


eqs = [D(k) ~ s*f(k) - (n + g + δ)*k] 

@named sys = ODESystem(eqs, t);

function ODE_rhs(k, α = alpha, δ = delta, n = nn, g = gg, s = ss)
    return s*f(k) - (n + g + δ)*k
end

steady_state = fzero(ODE_rhs, 0.0001, 1000)

tspan = (0.0, 100.0)
prob = ODEProblem(sys, [], tspan)

sol = solve(prob)

plot(sol)
hline!([steady_state], linecolor = :red, linewidth = 0.8, linestyle = :solid)

### Some takeaways

- The model has a steady state. It implies that in equilibrium absolute variables grow at a rate of population growth plus technology growth (plus a mixed term).
- The model is globally asymptotically stable
- Capital-poor economies can expect to grow and accumulate capital over time
- It is possible to have a process of "decapitalization" in effective per capita terms if initial capital is too high